<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [ ]:
# !mamba install pandas==1.3.3 -y
# !mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [1]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2022-10-10 09:49:09--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-10-10 09:49:10 (76.8 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2022-10-10 09:49:11--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [110]:
# Function that extracts data from a json file
# and stores the data into a pandas dataframe
def extract_from_json(file_to_process):
    
    """
    Extracts data from a json file and stores the data into a pandas dataframe
    ----------
    argument:
    file_to_process: json file to process in order to extract data
    ----------
    return:
    dataframe: pandas dataframe containing data from json file
    
    """
    
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [111]:
# Define the list of the columns for the dataframe to be created
columns = ['Name', 'Market Cap (US$ Billion)']

In [112]:
# Function that extracts data from bank_market_cap_1.json
# and stores the data in the pandas dataframe extracted_data
def extract(columns, json_file):
    
    """
    Extracts data from a json file and stores the data into a pandas dataframe
    with columns names from the list 'columns'
    ----------
    arguments:
    columns: list containing column names of the dataframe to be created
    json_file (string): json file to process in order to extract data
    ----------
    return:
    bank_market_cap: pandas dataframe containing data from json file
    
    """
    
    # Create an empty pandas dataframe with column names
    bank_market_cap = pd.DataFrame(columns=columns)
    # Add data from bank_market_cap_1.json into extracted_data
    bank_market_cap = bank_market_cap.append(
        extract_from_json(json_file),
        ignore_index=True
    )
    
    return bank_market_cap

In [113]:
# Extract data from bank_market_cap_1.json
bank_market_cap = extract(
    columns=['Name', 'Market Cap (US$ Billion)'],
    json_file='bank_market_cap_1.json'
)
# Print the first 5 rows
bank_market_cap.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [114]:
# Load the file exchange_rates.csv
exchange_rates = pd.read_csv('exchange_rates.csv', index_col=0)

In [115]:
# Display first rows
exchange_rates.head()

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083


In [116]:
# Find the exchange rate for British pounds ('GBP')
exchange_rate = exchange_rates.loc['GBP', 'Rates']
exchange_rate

0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [117]:
# Changes the Market Cap (US$ Billion) column from USD to GBP
bank_market_cap["Market Cap (US$ Billion)"] = bank_market_cap["Market Cap (US$ Billion)"] * exchange_rate
bank_market_cap.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621


In [118]:
# Rounds the Market Cap (US$ Billion)` column to 3 decimal places
bank_market_cap["Market Cap (US$ Billion)"] = round(bank_market_cap["Market Cap (US$ Billion)"], 3)
bank_market_cap.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [119]:
# Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)
bank_market_cap = bank_market_cap.rename(
    columns={
    'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'
}
                              )
bank_market_cap.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [120]:
# Transform function that changes the Market Cap (US$ Billion) column 
# from USD to GBP
def transform(exchange_rates_csv_file, bank_market_cap_USD):
    
    """
    Transforms the dataframe 'bank_market_cap' by  changing the Market Cap column
    from USD to GBP
    ----------
    arguments:
    exchange_rates_csv_file (string): csv file containing the exchange rates
    bank_market_cap_USD: pandas dataframe containing the Market Caps in US$ Billion
    ----------
    return:
    bank_market_cap_GBP: pandas dataframe containing the Market Caps in GB£ Billion
    
    """
    
    # Load the file exchange_rates.csv
    exchange_rates = pd.read_csv(exchange_rates_csv_file, index_col=0)
    # Find the exchange rate for British pounds ('GBP')
    exchange_rate = exchange_rates.loc['GBP', 'Rates']
    # Changes the Market Cap (US$ Billion) column from USD to GBP
    bank_market_cap_GBP = bank_market_cap_USD
    bank_market_cap_GBP["Market Cap (US$ Billion)"] = (
        bank_market_cap_GBP["Market Cap (US$ Billion)"] * exchange_rate
    )
    # Rounds the Market Cap (US$ Billion)` column to 3 decimal places
    bank_market_cap_GBP["Market Cap (US$ Billion)"] = round(
        bank_market_cap_GBP["Market Cap (US$ Billion)"], 
        3
    )
    # Rename Market Cap (US$ Billion) to Market Cap (GBP Billion)
    bank_market_cap_GBP = bank_market_cap_GBP.rename(
        columns={
            'Market Cap (US$ Billion)': 'Market Cap (GBP Billion)'
        }
    )
    
    return bank_market_cap_GBP

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [121]:
# Function that loads the dataframe into a csv file
def load(bank_market_cap, csv_file_name):
    
    """
    Loads the dataframe into a csv file
    ----------
    arguments:
    bank_market_cap: pandas dataframe containing the Market Caps
    csv_file_name (string): name of the csv file to be saved
    
    """
    
    bank_market_cap.to_csv(csv_file_name, index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [122]:
# Function that logs the data
def log(message):
    
    """
    Logs the data
    ----------
    arguments:
    message (string): message to be displayed
    
    """
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt", "a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [123]:
# Log the process: Start of ETL Job
log("ETL Job Started")

### Extract


In [124]:
# Log the process: Start of Extract
log("Extract phase Started")

<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [125]:
# Extract data from bank_market_cap_1.json
bank_market_cap = extract(
    columns=['Name', 'Market Cap (US$ Billion)'],
    json_file='bank_market_cap_1.json'
)

# Print the first 5 rows
bank_market_cap.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [126]:
# Log the process: End of Extract
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [127]:
# Log the process: Start of Transform
log("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [128]:
# Changes the Market Cap (US$ Billion) column from USD to GBP
bank_market_cap_GBP = transform(
    exchange_rates_csv_file='exchange_rates.csv',
    bank_market_cap_USD=bank_market_cap
)

# Print the first 5 rows
bank_market_cap_GBP.head()

,Name,Market Cap (GBP Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log your data <code>"Transform phase Ended"</code>


In [129]:
# Log the process: End of Transform
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [130]:
# Log the process: Start of Load
log("Load phase Started")

Call the load function


In [131]:
# Loads the dataframe into a csv file
load(
    bank_market_cap=bank_market_cap_GBP, 
    csv_file_name='bank_market_cap_gbp.csv'
)

Log the following `"Load phase Ended"`.


In [132]:
# Log the process: End of Load
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
